In [1]:
import requests
from config import req_headers, base_url, focus_league, local_host, local_name, local_port, local_user
from creds import local_db_password
import psycopg2
import time

In [2]:
conn = psycopg2.connect(dbname=local_name, user =local_user, host=local_host, password=local_db_password, port =local_port)
cur = conn.cursor()

In [3]:
endpoint = 'fixtures'
url = base_url + endpoint
season = 2023

call_params = {
    'league':focus_league,
    'season': season
}

In [4]:
def parser(data=None):
    id = data['fixture']['id']
    date = data['fixture']['date'][:10]
    # season = season
    team_home = data['teams']['home']['id']
    team_away = data['teams']['away']['id']
    tracking_status = 'posted'
    referee = data['fixture']['referee']
    s_home = data['goals']['home']
    s_away = data['goals']['away']

    return id, date, season, team_home, team_away, tracking_status, referee, s_home, s_away

In [5]:
req = requests.get(url,params=call_params, headers=req_headers)

response = req.json()['response']

In [6]:
sql = """
insert into ffl.fixtures values (%s, %s, %s, %s, %s, %s, %s, %s, %s) on conflict do nothing
"""

In [7]:
for row in response:
    cur.execute(sql, parser(row))
conn.commit()